In [45]:
import pandas as pd
import numpy as np
import re
import difflib
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


Open the data set. Use ISO encoding, for the pound sterling symbol.

In [46]:
welcome = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv',encoding='iso8859_15')
welcome.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


def  similarities(col1, col2):
 
    return process.extract(col1, col2)[0]

Look at types

In [3]:
welcome.dtypes

PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object

Look at unique data in each column.

In [47]:
welcome['PMID/PMCID'].to_csv('PMID.csv', index=False)

In [7]:
welcome['Publisher'].value_counts()

Elsevier                                                   387
Public Library of Science                                  278
Wiley                                                      136
Springer                                                    81
Oxford University Press                                     77
Wiley-Blackwell                                             56
OUP                                                         56
ASBMB                                                       46
Nature Publishing Group                                     45
BioMed Central                                              40
BMC                                                         26
Nature                                                      24
Frontiers                                                   23
BMJ                                                         23
Royal Society                                               22
Cambridge University Press                             

In [8]:
welcome['Journal title'].value_counts()

PLoS One                                                      92
PLoS ONE                                                      62
Journal of Biological Chemistry                               48
Nucleic Acids Research                                        21
Proceedings of the National Academy of Sciences               19
PLoS Neglected Tropical Diseases                              18
Human Molecular Genetics                                      18
Nature Communications                                         17
PLoS Genetics                                                 15
PLoS Pathogens                                                15
Neuroimage                                                    15
BMC Public Health                                             14
PLOS ONE                                                      14
Brain                                                         14
NeuroImage                                                    14
Movement Disorders       

In [9]:
#welcome['Article title'].unique()

In [10]:
welcome['COST (£) charged to Wellcome (inc VAT when charged)'].unique()

array(['£0.00', '£2381.04', '£642.56', ..., '£2015.72', '£1334.15',
       '£2034.75'], dtype=object)

Rename column 'COST (£) charged to Wellcome (inc VAT when charged)' to 'cost'. 
Rename other columns to one string to make copy and paste faster.

In [11]:
welcome.rename(index=str, columns={'PMID/PMCID':'PMID_PMCID',\
              'Journal title':'journal_title', \
              'Article title':'article_title', \
              'COST (£) charged to Wellcome (inc VAT when charged)':'cost'}, 
               inplace=True)

Remove the sterling pound symbol.

In [12]:
welcome.cost = welcome.cost.apply(lambda x: x.strip('£'))

Looking at unique values (above) there are instances of tailing white space.  
Strip the occasional white space from all columns.
First, replace empty NaN with empty strings.

In [13]:
welcome = welcome.replace(np.nan, '', regex=True)

In [14]:
welcome.PMID_PMCID = welcome.PMID_PMCID.apply(lambda x: x.strip())
welcome.Publisher = welcome.Publisher.apply(lambda x: x.strip())
welcome.journal_title = welcome.journal_title.apply(lambda x: x.strip())
welcome.article_title = welcome.article_title.apply(lambda x: x.strip())
welcome.cost = welcome.cost.apply(lambda x: x.strip())


Some publishers are in journal title, look up journal by PMID and replace.

pd.set_option('display.max_colwidth', -1)

welcome[welcome['journal_title']=='Scientific Reports']

#welcome_2 = pd.DataFrame(welcome)
welcome.iloc[546, 2] = 'Brain Behav Immun'
welcome.iloc[547, 2] = 'Behav Res Ther'
welcome.iloc[548, 2] = 'J Pediatr'
welcome.iloc[549, 2] = 'Pathogens Dis'
welcome.iloc[550, 2] = 'Health Place'
welcome.iloc[551, 2] = 'Drug Alcohol Depend'
welcome.iloc[552, 2] = 'Lancet'
welcome.iloc[1255, 2] = 'Health Policy Plan'
welcome.iloc[1256, 2] = 'Age Aging'
welcome.iloc[1793, 2] = 'Acta Neuropathol'
welcome.iloc[1614, 2] = 'PLoS Medicine'
welcome.iloc[1616, 2] = 'PLoS One' 
welcome.iloc[1615, 2] = 'PLoS Negl Trop Dis'
welcome.iloc[1343, 2] = 'PLoS Negl Trop Dis'
welcome.iloc[1344, 2] = 'PLoS Negl Trop Dis'
welcome.iloc[986, 2] = 'Mycoses and Modernity'
welcome.iloc[1643, 2] = 'Proc Biol Sci'
welcome.iloc[1642, 2] = 'Proc Biol Sci'
welcome.iloc[1860, 2] = 'Proc Biol Sci'
welcome.iloc[1644, 2] = 'Proc Biol Sci'
welcome.iloc[1647, 2] = 'Proc Biol Sci'
welcome.iloc[1648, 2] = 'Proc Biol Sci'
welcome.iloc[1649, 2] = 'Proc Biol Sci'
welcome.iloc[1861, 2] = 'Proc Biol Sci'
welcome.iloc[1862, 2] = 'Proc Biol Sci'
welcome.iloc[1863, 2] = 'Proc Biol Sci'
welcome.iloc[1818, 2] = 'Eur J Engl Stud'
welcome.iloc[1817, 2] = 'Engl Stud (Amst)'
welcome.iloc[1859, 2] = 'Philos Trans R Soc Lond B Biol Sci'
welcome.iloc[1864, 2] = 'Philos Trans R Soc Lond B Biol Sci'
welcome.iloc[115, 2] = 'J Biol Chem'
welcome.iloc[153, 2] = 'Mol Cell Proteomics'
welcome.iloc[659, 2] = 'Molecular and Biochemical Parasitology'
welcome.iloc[1258, 2] = 'QJM'
welcome.iloc[1259, 2] = 'QJM'
welcome.iloc[1711, 2] = 'Acta Derm Venereol'
welcome.iloc[1716, 2] = 'Acta Diabetologica'
welcome.iloc[1800, 2] = 'Reprod Biol Endocrinol'
welcome.iloc[1856, 2] = 'Br J Psychiatry'
welcome.iloc[2079, 2] = 'Journal of the International Hepato Pancreato Biliary Association'
welcome.iloc[176, 2] = 'J Neuroinflammation'
welcome.iloc[85, 2] = 'Antimicrob Agents Chemother'
welcome.iloc[1057, 2] = 'EMBO J'
welcome.iloc[920, 2] = 'Acta Crystallogr D Biol Crystallogr'
welcome.iloc[921, 2] = 'Acta Crystallogr D Biol Crystallogr'
welcome.iloc[922, 2] = 'Acta Crystallogr D Biol Crystallogr'
welcome.iloc[923, 2] = 'Acta Crystallogr D Biol Crystallogr'
welcome.iloc[927, 2] = 'Acta Crystallogr D Biol Crystallogr'
welcome.iloc[924, 2] = 'Acta Crystallogr Sect F Struct Biol Cryst Commun'
welcome.iloc[928, 2] = 'Acta Crystallogr Sect F Struct Biol Cryst Commun'
welcome.iloc[929, 2] = 'Acta Crystallogr Sect F Struct Biol Cryst Commun'
welcome.iloc[455, 2] = 'Biochim Biophys Acta'
welcome.iloc[464, 2] = 'Biochim Biophys Acta'
welcome.iloc[465, 2] = 'Biochim Biophys Acta'
welcome.iloc[467, 2] = 'Biochim Biophys Acta'
welcome.iloc[467, 2] = 'Biochim Biophys Acta'
welcome.iloc[787, 2] = 'Stud Hist Philos Biol Biomed Sci'
welcome.iloc[788, 2] = 'Stud Hist Philos Biol Biomed Sci'
welcome.iloc[803, 2] = 'Trans R Soc Trop Med Hyg'
welcome.iloc[918, 2] = 'Int J Tuberc Lung Dis'
welcome.iloc[1097, 2] = 'Sci Rep'
welcome.iloc[1048, 2] = 'Sci Rep'
welcome.iloc[1049, 2] = 'Sci Rep'
welcome.iloc[1096, 2] = 'Sci Rep'

welcome.iloc[918]

Clean up journal titles.  

j_titles = pd.DataFrame(welcome.journal_title.unique())
j_titles.rename(index=int, columns={0:'title'},inplace=True)
j_titles.to_csv('titles.csv')

#{'\\.':'',{',':'', ':':'', '-':'',}

j_titles.title = j_titles.title.str.lower()
fixes =   '(\\w*the\\w*)(\\s)':''}#,\
         #'(\\w*for\\w*)(\\s)':'', 'joural':'journal', 'jounral':'journal',\
         #'journ.*? of':'j ', 'jnl':'j', '(\\b)(\\w*journ.*?\\w*)(\\b)':'j',\
         #'(\\b)(\\w*online\\w*)(\\b)':'', 'american chemical society':'acs',\
         #'(\\b)(\\w*proceedings.*? sciences\\w*)(\\b)':'pnas',\
         #'bmc':'BioMed Cntrl', 'bms':'BioMed Cntrl', 'bmj':'Br Med J',\
         #'antimicrobial agents and chemotherapy':'Antimicrob Agents Chemother',\
         #'national academy of sciences':'pnas', \
         #'pnas \\(pnas of united states of america':'pnas', \
         #'proceddings of pnas of usa':'pnas', 'of united states of america':'',\
         #'of usa':'', 'usa':'', 'proc natl acad sci u s a':'pnas',\
         #'pnas \\(pnas\\)':'pnas', 'jaids':'', \
         #'jaids j acquired immune deficiency syndromes':'j aids',\
         #'\\(\\)':'', '\\(':'', '\\)':'',\
         #'acquired immune deficiency syndroms':'aids',\
         #'acquired immune deficiency syndromes':'aids', 'aids j':'aids',\
         #'aids uk':'aids', '&':'and', 'amer.*? ':'am ', 'europe.*? ':'eu ',\
         #'(\\b)(\\w*brit.*?\\w*)(\\b)':'br', 'brt':'br', 'english':'en',\
         #'canadian':'cn', '(\\b)(\\w*medi.*?\\w*)(\\b)':'med', \
         #'(\\b)(\\w*internat.*?\\w*)(\\b)':'intl ',\
         #'int j epidemiol':'intl j epidemiol', \
         #'(\\b)(\\w*chemi.*?\\w*)(\\b)':' chem', 'ogy':'', \
         #'ogical':'', 'ogicial':'', 'iatry':'', 'research':'resch',\
         #'(\\b)(\\w*neuroscience\\w*)(\\b)':'neurocsi', \
         #'(\\b)(\\w*behavi\\w*)(\\b)':'behav', 'sical':'', 'ivity':'',\
         #'molecular':'mol', '(\\b)(\\w*cellul\\w*)(\\b)':'cell',\
         #'(\\b)(\\w*proteomi\\w*)(\\b)':'proteo', \
         #'(\\b)(\\w*current\\w*)(\\b)':'curr', \
         #'(\\b)(\\w*topics\\w*)(\\b)':'top', \
         #'(\\b)(\\w*respir\\w*)(\\b)':'respir ',\
         #'reproduction\\(cambridge, england\\)':'repoduct', \
         #'(\\b)(\\w*chemotherapy\\w*)(\\b)':'chemother',\
         #'(\\b)(\\w*psychogeri\\w*)(\\b)':'psychogeri', \
         #'(\\b)(\\w*nutrit\\w*)(\\b)':'nutri', \
         #'(\\b)(\\w*institute\\w*)(\\b)':'inst', 'cell press -':'',\
         #'(\\b)(\\w*respi.*?\\w*)(\\b)':'respir ', \
         #'(\\b)(\\w*opinion\\w*)(\\b)':'opinions', '(\\s)(in)(\\s)':' ',\
         #'(\\b)(\\w*neuroimage\\w*)(\\b)':'neuroImage', \
         #'biinformatics':'bioinformatics', \
         #'nucleic acids resch':'nucleic acid resch', \
         #'public library of science':'plos', 'plos one':'plos',\
         #'plos  one':'plos', 'plos 1':'plos', 'plosone':'plos', \
         #'plos':'plos one', 'negected':'neglected', \
         #'plos one med j':'plos one med', \
         #'j autism and development disorders':'j autism and developmental disorders',\
         #'(\\b)(\\w*archiv\\w*)(\\b)':'archive', 'scienceresch':'science resch',\
         #'applied statistics':'', 'blood j 2012':'blood', 'opthalmol':'ophthalmol',\
         #'bjophthalmol':'br j ophthalmol', 'mol bio':'mol biol', \
         #'epigentics':'epigenetics', 'expermiments':'experiments', \
         #'sci rep':'scientific reports', \
         #'nephrol dialysis transplantation':'nephrol dialysis and transplantation'}
#{'aids':AIDS', '(\\b)(\\w*AIDS j\\w*)(\\b)':'AIDS', 'AIDS uk':'AIDS'}

j_titles.title = j_titles.title.replace(fixes, inplace=True, regex=True)



pd.set_option('display.max_colwidth', -1)
j_titles.iloc[850:900]
#mod_j_titles = pd.DataFrame(j_titles.title.unique())
#mod_j_titles.to_csv('titles_2.csv')


mod_j_titles = pd.DataFrame(j_titles.title.unique())

#.str.replace('chembiochem', 'ChemBioChem')\
#.str.replace('chemmedchem', 'ChemMedChem')\


mod_j_titles[50:100]

In [81]:
pmid = pd.read_csv('PMID.csv')
pmid = pmid.replace(np.nan, '', regex=True)

pmid.rename(index=str, columns={'Unnamed: 0':'id'}, inplace=True)
pmid.id = pmid.id.str.strip()
replacements = {': ':' ', ',':' ', '(\r\n)':' ', '(\\D)':' '}
pmid.id = pmid.id.replace(replacements, regex=True)
pmid['first_part'], pmid['second_part'] = pmid.id.str.split(' ', 1).str
pmid.to_csv('numbers.csv', index=False)
pmid[0:50]

,id,first_part,second_part
0,3679557,,3679557
1,23043264 3506128,23043264,3506128
2,23438330 3646402,23438330,3646402
3,23438216 3601604,23438216,3601604
4,3579457,,3579457
5,3709265,,3709265
6,23057412 3495574,23057412,3495574
7,3780468,,3780468
8,3621575,,3621575
9,3739413,,3739413
